In [ ]:
import sys
from pathlib import Path
from copy import deepcopy
import torch
import matplotlib.pyplot as plt
import numpy as np

try:
    if not(changed):
        raise Exception()
except:
    sys.path.append(str(Path(".").absolute().parent))
    changed = True

%load_ext autoreload
%autoreload 2

In [ ]:
from notebooks.notebooks_utils import get_activation, load_checkpoint

job_id = "4344494" # faenet

trainer = load_checkpoint(job_id)
train_loader = trainer.get_dataloader(trainer.datasets["train"], trainer.samplers["train"])

In [ ]:
from notebooks.notebooks_utils import oc20_to_graph, plot_element_3d, process_datapoint

In [ ]:
from ocpmodels.modules.evaluator import Evaluator
import seaborn as sns

In [ ]:
val_loader = trainer.get_dataloader(trainer.datasets["val_id"], trainer.samplers["val_id"])
trainer.model.eval()
evaluator = Evaluator(
    task=trainer.task_name,
    model_regresses_forces="",
)

batch = next(iter(val_loader))
maes = []
for _ in range(1000):
    with torch.cuda.amp.autocast(enabled=trainer.scaler is not None):
        preds = energy_prediction = trainer.model_forward(batch, mode="val")
    metrics = trainer.compute_metrics(preds, batch, evaluator, metrics={})
    maes.append(metrics["energy_mae"]["metric"])
sns.displot(maes, kde=True)
plt.title("Distribution of MAE for energy prediction on the same element")
plt.show()

In [ ]:
import os
import pickle

In [ ]:
directory = "/network/scratch/s/schmidtv/ocp/datasets/ocp/per_ads/oc20_data_mapping.pkl"
metadata = pickle.load(open(directory, "rb"))

In [ ]:
val_metrics = trainer.validate("val_id", disable_tqdm=False)

In [ ]:
from collections import defaultdict
from tqdm import tqdm
import networkx as nx

pbar = tqdm(total=len(val_loader))
scores = defaultdict(list)
mispredicted_elements = []
mispredicted_indices = [] 

for i, batch in enumerate(val_loader):
    with torch.cuda.amp.autocast(enabled=trainer.scaler is not None):
        preds = energy_prediction = trainer.model_forward(batch, mode="val")
    metrics = trainer.compute_metrics(preds, batch, evaluator, metrics={})
    mae = metrics["energy_mae"]["metric"]
    scores["mae"].append(mae)
    pbar.set_description(f"MAE: {mae}")
    pbar.update(1)
    pbar.refresh()

    data_graph = oc20_to_graph(batch[0], processed=False)
    try:
        scores["diameter"].append(nx.diameter(data_graph))
    except:
        scores["diameter"].append(0)
    scores["connected"].append(nx.is_connected(data_graph))
    scores["number_of_nodes"].append(data_graph.number_of_nodes())
    scores["number_of_edges"].append(data_graph.number_of_edges())
    scores["edges/nodes"].append(data_graph.number_of_edges() / data_graph.number_of_nodes())
    # scores["density"].append(nx.density(data_graph))
    scores["target"].append(batch[0].y_relaxed.cpu().numpy()[0])
    scores["n_surface"].append((batch[0].tags.cpu().numpy() == 1).sum())
    scores["n_adsorbate"].append((batch[0].tags.cpu().numpy() == 2).sum())
    scores["y_init"].append(batch[0].y_init.cpu().numpy()[0])
    scores["average_degree"].append(np.mean(list(dict(data_graph.degree()).values())))
    scores["ads_symbols"].append(metadata[f"random{batch[0].sid.cpu().numpy()[0]}"]["ads_symbols"])
    scores["ads_id"].append(metadata[f"random{batch[0].sid.cpu().numpy()[0]}"]["ads_id"])
    scores["anomaly"].append(metadata[f"random{batch[0].sid.cpu().numpy()[0]}"]["anomaly"])
    scores["bulk_symbols"].append(metadata[f"random{batch[0].sid.cpu().numpy()[0]}"]["bulk_symbols"])
    scores["bulk_id"].append(metadata[f"random{batch[0].sid.cpu().numpy()[0]}"]["bulk_id"])
    scores["shift"].append(metadata[f"random{batch[0].sid.cpu().numpy()[0]}"]["shift"])


    if mae > 10:
        mispredicted_elements.append(batch[0])
        mispredicted_indices.append(i)


pbar.close()
sns.displot(scores["mae"], kde=True)
plt.title("Distribution of MAE for energy prediction on different elements")
plt.show()

In [ ]:
n = 7

fig, axes = plt.subplots(n, 3, figsize=(10, 20))

scores["edges + nodes"] = np.array(scores["number_of_edges"]) + np.array(scores["number_of_nodes"])
scores['y_init - target'] = np.array(scores["y_init"]) - np.array(scores["target"])
scores["abs_target"] = np.abs(np.array(scores["target"]))
scores["connected"] = np.array(scores["connected"]).astype(int)

for i, key in enumerate(scores.keys()):
    ax = axes[i // 3, i % 3]
    if key not in [] and "symbol" not in key:
        ax.hist(scores[key])
    ax.set_title(key)

fig.tight_layout()
plt.show()

fig, axes = plt.subplots(n, 3, figsize=(10, 20))

for i, key in enumerate(scores.keys()):
    ax = axes[i // 3, i % 3]
    if key not in [] and "symbol" not in key:
        ax.plot(scores[key], scores["mae"], "o")
        ax.set_title(f"{key} - Pearson: {np.corrcoef(scores[key], scores['mae'])[0, 1]:.2f}")
    ax.set_ylabel("MAE")
    ax.set_xlabel(key)

fig.tight_layout()
plt.show()

In [ ]:
%matplotlib inline
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

threshold = 0
linreg = Ridge(alpha=1e-3)
X_list = []
keys = []
for key in scores.keys():
    if key not in ["mae"] and "symbols" not in key:
    # if key not in ["mae", "y_init", "target", "y_init - target"] and "symbol" not in key:
        X_list.append(scores[key][scores["mae"] > threshold].astype(float))
        keys.append(key)

X = np.stack(X_list, axis=1)
y = scores["mae"][scores["mae"] > threshold]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

linreg.fit(X_train, y_train)
y_pred = linreg.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred_tree = xgb_reg.predict(X_test)
mae_tree = mean_absolute_error(y_test, y_pred_tree)
mse_tree = mean_squared_error(y_test, y_pred_tree)
print(f"Linear regression: MAE: {mae:.2f}, MSE: {mse:.2f}")

plt.plot(y_test, y_pred, "o")
plt.xlabel("True MAE")
plt.ylabel("Predicted MAE")
plt.title(f"MAE: {mae:.2f}, MSE: {mse:.2f}")
plt.show()

plt.barh(keys, linreg.coef_)
plt.xlabel("Coefficient")
plt.ylabel("Feature")
plt.title("Linear regression coefficients")
plt.show()


plt.plot(y_test, y_pred_tree, "o")
plt.xlabel("True MAE")
plt.ylabel("Predicted MAE")
plt.title(f"MAE: {mae_tree:.2f}, MSE: {mse_tree:.2f}")
plt.show()

plt.barh(keys, xgb_reg.feature_importances_)
plt.xlabel("Feature importance")
plt.ylabel("Feature")
plt.title("XGBoost feature importances")
plt.show()
print(list(zip(xgb_reg.feature_importances_, keys)))

print(list(zip(linreg.coef_, keys)), linreg.intercept_)

In [ ]:
# box plot of mae on different bulk_id

keys = ["bulk_id", "ads_id"]
symbols_keys = ["bulk_symbols", "ads_symbols"]

for key, symbols_key in zip(keys, symbols_keys):
    n_bulks = min(80, len(np.unique(scores[key])))
    bulk_ids = np.array(scores[key])
    bulk_symbols = np.array(scores[symbols_key])
    unique_bulk_ids = np.unique(bulk_ids)[:n_bulks]
    bulk_ids_index = np.zeros_like(bulk_ids).astype(bool)
    bulk_symbols_plot = np.zeros(n_bulks).astype(str)
    for i, bulk_id in enumerate(unique_bulk_ids):
        bulk_ids_index[bulk_ids == bulk_id] = True
        bulk_symbols_plot[i] = bulk_symbols[bulk_ids == bulk_id][0]
    bulk_ids_plot = np.array(bulk_ids)[bulk_ids_index]
    maes = np.array(scores["mae"])[bulk_ids_index]

    fig, ax = plt.subplots(figsize=(20, 5))
    ax.boxplot([maes[bulk_ids_plot == bulk_id] for bulk_id in unique_bulk_ids])
    ax.set_xticklabels(bulk_symbols_plot)
    ax.set_title(f"MAE distribution for different {key}")
    ax.set_xlabel(key)
    ax.set_ylabel("MAE")
    plt.show()